In [ ]:

## C2 Beacons found at 192.168.120.158 on port 443 communicating with 192.168.128.50


In [1]:
#Elasticsearch connector
from elasticsearch import Elasticsearch
from elasticsearch_dsl import Search
import eland as ed
#Data manipulation
import pandas as pd
from pandas.io.json import json_normalize
import json
import re
import eland as ed
import numpy as np
import matplotlib.pyplot as plt
import math
from datetime import datetime
# Disable warnings"
import urllib3
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)
import warnings
warnings.filterwarnings('ignore')
warnings.simplefilter('ignore')

In [2]:
es = Elasticsearch(['https://192.168.1.181:9200'], timeout=30, max_retries=10, retry_on_timeout=True,                  
ca_certs=False,verify_certs=False, http_auth=('jupyter','password'))

# defining the search statement to get all records in an index
searchContext = Search(using=es, index='*:so-*', doc_type='doc')
es.info()

ObjectApiResponse({'name': 'voodoo-onion', 'cluster_name': 'voodoo-onion', 'cluster_uuid': 'dP1VycCSTQ2EPeCVxQ25GA', 'version': {'number': '7.16.2', 'build_flavor': 'default', 'build_type': 'docker', 'build_hash': '2b937c44140b6559905130a8650c64dbd0879cfb', 'build_date': '2021-12-18T19:42:46.604893745Z', 'build_snapshot': False, 'lucene_version': '8.10.1', 'minimum_wire_compatibility_version': '6.8.0', 'minimum_index_compatibility_version': '6.0.0-beta1'}, 'tagline': 'You Know, for Search'})

In [3]:
query_since_ts = 0

#body_filter = '{"query":{"bool":{"filter":[{"range":{@timestamp":{"gt":'+ str(query_since_ts) + '}}}]}},"sort":[{"@timstamp":{"order":"asc"}}]}'

body_filter = '{"query":{"@timestamp":{}},"size=10000":[{"@timestamp":{"order":"asc"}}]}'

In [9]:
#Firewall 192.168.1.154

s = searchContext.query('query_string', query='(event.category:(network OR network_traffic) AND network.transport:tcp) AND destination.port:(53 OR 80 OR 8080 OR 443) AND (@timestamp > "now-3d/d") AND NOT source.ip:192.168.1.154')


In [10]:
pd.set_option("display.max_rows", 100)
pd.set_option("max_colwidth", 600)
pd.set_option("display.max_columns", None)


In [11]:
response = s.execute()
df = pd.DataFrame(columns=['ts','source.ip','destination.ip','destination.port'])
if response.success():
    for d in s[:10000]:
        try:
            df = df.append({'ts' : d['@timestamp'], 'source.ip' : d['source']['ip'], 'destination.ip' : d['destination']['ip'], 'destination.port': d['destination']['port']}, ignore_index=True)
        except KeyError:
            pass       

display(df)

df.info()

,ts,source.ip,destination.ip,destination.port
0,2021-12-29T22:20:52.526Z,192.168.128.15,23.66.101.177,80
1,2021-12-29T22:20:53.091Z,192.168.128.15,23.66.101.177,80
2,2021-12-29T22:20:52.576Z,192.168.128.15,23.66.101.177,80
3,2021-12-29T22:20:54.325Z,192.168.128.15,23.66.101.177,80
4,2021-12-29T22:20:52.593Z,192.168.128.15,23.66.101.177,80
...,...,...,...,...
3097,2022-01-01T07:36:44.472Z,192.168.128.26,168.63.250.82,80
3098,2022-01-01T07:36:44.171Z,192.168.128.26,168.63.250.82,80
3099,2022-01-01T07:36:43.891Z,192.168.128.26,168.63.250.82,80
3100,2022-01-01T07:36:43.584Z,192.168.128.26,168.63.250.82,80


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3102 entries, 0 to 3101
Data columns (total 4 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   ts                3102 non-null   object
 1   source.ip         3102 non-null   object
 2   destination.ip    3102 non-null   object
 3   destination.port  3102 non-null   object
dtypes: object(4)
memory usage: 97.1+ KB


In [12]:
# assign field/column names to variables
f_timestamp = 'ts'
f_src_ip = 'source.ip'
f_dst_ip = 'destination.ip'
f_dst_port = 'destination.port'
f_delimiter = '\t'


In [13]:
df['ts'] = pd.to_datetime(df['ts'],infer_datetime_format=True)
df.head(5)

,ts,source.ip,destination.ip,destination.port
0,2021-12-29 22:20:52.526000+00:00,192.168.128.15,23.66.101.177,80
1,2021-12-29 22:20:53.091000+00:00,192.168.128.15,23.66.101.177,80
2,2021-12-29 22:20:52.576000+00:00,192.168.128.15,23.66.101.177,80
3,2021-12-29 22:20:54.325000+00:00,192.168.128.15,23.66.101.177,80
4,2021-12-29 22:20:52.593000+00:00,192.168.128.15,23.66.101.177,80


In [14]:

columns_to_filter = [f_timestamp, f_src_ip, f_dst_ip, f_dst_port,]
columns_to_groupby = [f_src_ip, f_dst_ip, f_dst_port,]
# columns to display after the analysis
columns_to_display = ['tsScore','conn_count',f_src_ip,f_dst_ip,f_dst_port,'deltas']

In [15]:
# If you have a large dateset, using groupby and aggregate(list) might be slow. Consider using dask
df = df.groupby(columns_to_groupby).agg(list)
df.head(50)

ts
source.ip      destination.ip destination.port                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                         
192.168.128.10 104.208.16.90  443                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                    [2021-12-29 23:38:57.271000+00:00]
               13.78.111.198  443                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                    [2021-12-29 21:38:56.996000+00:00]
               20.189.173.10  443                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  [2021-12-29 22:38:57.005000+00:00, 2021-12-29 22:08:56.888000+00:00]
               20.189.173.13  443                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                    [2021-12-29 22:59:06.962000+00:00]
               20.42.73.25    443                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  [2021-12-29 22:43:42.279000+00:00, 2021-12-29 22:43:43.195000+00:00]
               20.50.80.209   443                                                                            

In [16]:
df.reset_index(inplace=True)
df.head(5)

,source.ip,destination.ip,destination.port,ts
0,192.168.128.10,104.208.16.90,443,[2021-12-29 23:38:57.271000+00:00]
1,192.168.128.10,13.78.111.198,443,[2021-12-29 21:38:56.996000+00:00]
2,192.168.128.10,20.189.173.10,443,"[2021-12-29 22:38:57.005000+00:00, 2021-12-29 22:08:56.888000+00:00]"
3,192.168.128.10,20.189.173.13,443,[2021-12-29 22:59:06.962000+00:00]
4,192.168.128.10,20.42.73.25,443,"[2021-12-29 22:43:42.279000+00:00, 2021-12-29 22:43:43.195000+00:00]"


In [17]:
# create a new column 'conn_count', and for each row in the 'ts' column, apply a function and assign the returned value to the 'conn_count' column
df['conn_count'] = df[f_timestamp].apply(lambda x: len(x))
df.head(2)

,source.ip,destination.ip,destination.port,ts,conn_count
0,192.168.128.10,104.208.16.90,443,[2021-12-29 23:38:57.271000+00:00],1
1,192.168.128.10,13.78.111.198,443,[2021-12-29 21:38:56.996000+00:00],1


In [18]:
#Filter out traffic where the connection count is quite small
df = df.loc[df['conn_count'] > 50]
df.shape

(5, 5)

In [19]:
df[f_timestamp] = df[f_timestamp].apply(lambda x: sorted(x))
df.head()

,source.ip,destination.ip,destination.port,ts,conn_count
38,192.168.128.15,23.66.101.177,80,"[2021-12-29 22:20:52.436000+00:00, 2021-12-29 22:20:52.447000+00:00, 2021-12-29 22:20:52.464000+00:00, 2021-12-29 22:20:52.498000+00:00, 2021-12-29 22:20:52.498000+00:00, 2021-12-29 22:20:52.503000+00:00, 2021-12-29 22:20:52.516000+00:00, 2021-12-29 22:20:52.526000+00:00, 2021-12-29 22:20:52.552000+00:00, 2021-12-29 22:20:52.576000+00:00, 2021-12-29 22:20:52.576000+00:00, 2021-12-29 22:20:52.588000+00:00, 2021-12-29 22:20:52.593000+00:00, 2021-12-29 22:20:52.604000+00:00, 2021-12-29 22:20:52.621000+00:00, 2021-12-29 22:20:52.636000+00:00, 2021-12-29 22:20:52.651000+00:00, 2021-12-29 22:20:...",159
129,192.168.128.50,192.168.120.158,80,"[2021-12-30 12:56:41.494000+00:00, 2021-12-30 12:57:41.627000+00:00, 2021-12-30 12:58:41.787000+00:00, 2021-12-30 12:59:41.807000+00:00, 2021-12-30 13:00:41.824000+00:00, 2021-12-30 13:01:41.845000+00:00, 2021-12-30 13:02:41.870000+00:00, 2021-12-30 13:03:41.893000+00:00, 2021-12-30 13:04:41.919000+00:00, 2021-12-30 13:05:41.933000+00:00, 2021-12-30 13:06:42.024000+00:00, 2021-12-30 13:07:42.042000+00:00, 2021-12-30 13:08:42.055000+00:00, 2021-12-30 13:09:42.071000+00:00, 2021-12-30 13:10:42.097000+00:00, 2021-12-30 13:11:42.119000+00:00, 2021-12-30 13:12:42.142000+00:00, 2021-12-30 13:13:...",2036
130,192.168.128.50,192.168.120.158,443,"[2021-12-29 19:58:55.104000+00:00, 2021-12-29 20:00:55.398000+00:00, 2021-12-29 20:02:55.601000+00:00, 2021-12-29 20:04:55.813000+00:00, 2021-12-29 20:04:55.825000+00:00, 2021-12-29 20:06:56.198000+00:00, 2021-12-29 20:08:56.445000+00:00, 2021-12-29 20:10:56.640000+00:00, 2021-12-29 20:18:37.658000+00:00, 2021-12-29 20:18:57.668000+00:00, 2021-12-29 20:20:57.892000+00:00, 2021-12-29 20:22:58.180000+00:00, 2021-12-29 20:24:58.392000+00:00, 2021-12-29 20:26:58.640000+00:00, 2021-12-29 20:28:58.870000+00:00, 2021-12-29 20:30:59.125000+00:00, 2021-12-29 20:32:59.367000+00:00, 2021-12-29 20:34:...",72
132,192.168.128.50,192.168.120.158,443,"[2021-12-29 19:03:30.945000+00:00, 2021-12-29 19:05:31.420000+00:00, 2021-12-29 19:07:31.521000+00:00, 2021-12-29 19:09:31.822000+00:00, 2021-12-29 19:11:31.943000+00:00, 2021-12-29 19:13:32.214000+00:00, 2021-12-29 19:15:32.334000+00:00, 2021-12-29 19:17:32.665000+00:00, 2021-12-29 19:19:32.838000+00:00, 2021-12-29 19:21:32.975000+00:00, 2021-12-29 19:23:33.380000+00:00, 2021-12-29 19:25:33.518000+00:00, 2021-12-29 19:27:33.919000+00:00, 2021-12-29 19:29:33.990000+00:00, 2021-12-29 19:31:34.419000+00:00, 2021-12-29 19:33:34.643000+00:00, 2021-12-29 19:35:34.882000+00:00, 2021-12-29 19:37:...",97
176,192.168.128.50,8.240.196.126,80,"[2021-12-29 22:03:21.207000+00:00, 2021-12-29 22:03:21.246000+00:00, 2021-12-29 22:03:21.305000+00:00, 2021-12-29 22:03:21.331000+00:00, 2021-12-29 22:03:21.405000+00:00, 2021-12-29 22:03:21.444000+00:00, 2021-12-29 22:03:21.486000+00:00, 2021-12-29 22:03:21.523000+00:00, 2021-12-29 22:03:21.534000+00:00, 2021-12-29 22:03:21.588000+00:00, 2021-12-29 22:03:21.614000+00:00, 2021-12-29 22:03:21.670000+00:00, 2021-12-29 22:03:21.697000+00:00, 2021-12-29 22:03:21.750000+00:00, 2021-12-29 22:03:21.812000+00:00, 2021-12-29 22:03:21.817000+00:00, 2021-12-29 22:03:21.890000+00:00, 2021-12-29 22:03:...",159


In [20]:
# Convert list into a Series object, get time delta, convert the result back into a list and assign it to the 'deltas' column
df['deltas'] = df[f_timestamp].apply(lambda x: pd.Series(x).diff().dt.seconds.dropna().tolist())
df.head(2)

,source.ip,destination.ip,destination.port,ts,conn_count,deltas
38,192.168.128.15,23.66.101.177,80,"[2021-12-29 22:20:52.436000+00:00, 2021-12-29 22:20:52.447000+00:00, 2021-12-29 22:20:52.464000+00:00, 2021-12-29 22:20:52.498000+00:00, 2021-12-29 22:20:52.498000+00:00, 2021-12-29 22:20:52.503000+00:00, 2021-12-29 22:20:52.516000+00:00, 2021-12-29 22:20:52.526000+00:00, 2021-12-29 22:20:52.552000+00:00, 2021-12-29 22:20:52.576000+00:00, 2021-12-29 22:20:52.576000+00:00, 2021-12-29 22:20:52.588000+00:00, 2021-12-29 22:20:52.593000+00:00, 2021-12-29 22:20:52.604000+00:00, 2021-12-29 22:20:52.621000+00:00, 2021-12-29 22:20:52.636000+00:00, 2021-12-29 22:20:52.651000+00:00, 2021-12-29 22:20:...",159,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...]"
129,192.168.128.50,192.168.120.158,80,"[2021-12-30 12:56:41.494000+00:00, 2021-12-30 12:57:41.627000+00:00, 2021-12-30 12:58:41.787000+00:00, 2021-12-30 12:59:41.807000+00:00, 2021-12-30 13:00:41.824000+00:00, 2021-12-30 13:01:41.845000+00:00, 2021-12-30 13:02:41.870000+00:00, 2021-12-30 13:03:41.893000+00:00, 2021-12-30 13:04:41.919000+00:00, 2021-12-30 13:05:41.933000+00:00, 2021-12-30 13:06:42.024000+00:00, 2021-12-30 13:07:42.042000+00:00, 2021-12-30 13:08:42.055000+00:00, 2021-12-30 13:09:42.071000+00:00, 2021-12-30 13:10:42.097000+00:00, 2021-12-30 13:11:42.119000+00:00, 2021-12-30 13:12:42.142000+00:00, 2021-12-30 13:13:...",2036,"[60.0, 60.0, 60.0, 60.0, 60.0, 60.0, 60.0, 60.0, 60.0, 60.0, 60.0, 60.0, 60.0, 60.0, 60.0, 60.0, 60.0, 60.0, 60.0, 60.0, 60.0, 60.0, 60.0, 60.0, 60.0, 60.0, 60.0, 60.0, 60.0, 60.0, 60.0, 60.0, 60.0, 15484.0, 60.0, 0.0, 5721.0, 60.0, 0.0, 60.0, 60.0, 0.0, 59.0, 0.0, 59.0, 0.0, 65.0, 54.0, 10.0, 65.0, 44.0, 20.0, 65.0, 34.0, 30.0, 65.0, 24.0, 40.0, 65.0, 14.0, 50.0, 65.0, 4.0, 60.0, 59.0, 5.0, 65.0, 49.0, 15.0, 65.0, 39.0, 25.0, 65.0, 29.0, 35.0, 65.0, 19.0, 45.0, 65.0, 9.0, 55.0, 64.0, 0.0, 65.0, 54.0, 10.0, 65.0, 44.0, 20.0, 65.0, 1451.0, 30.0, 65.0, 24.0, 40.0, 65.0, 14.0, 40.0, 9.0, 27.0..."


In [21]:
df['tsLow'] = df['deltas'].apply(lambda x: np.percentile(np.array(x), 20))
df['tsMid'] = df['deltas'].apply(lambda x: np.percentile(np.array(x), 50))
df['tsHigh'] = df['deltas'].apply(lambda x: np.percentile(np.array(x), 80))
df['tsBowleyNum'] = df['tsLow'] + df['tsHigh'] - 2*df['tsMid']
df['tsBowleyDen'] = df['tsHigh'] - df['tsLow']
df['tsSkew'] = df[['tsLow','tsMid','tsHigh','tsBowleyNum','tsBowleyDen']].apply(
    lambda x: x['tsBowleyNum'] / x['tsBowleyDen'] if x['tsBowleyDen'] != 0 and x['tsMid'] != x['tsLow'] and x['tsMid'] != x['tsHigh'] else 0.0, axis=1
    )
df['tsMadm'] = df['deltas'].apply(lambda x: np.median(np.absolute(np.array(x) - np.median(np.array(x)))))
df['tsConnDiv'] = df[f_timestamp].apply(lambda x: (x[-1].to_pydatetime() - x[0].to_pydatetime()).seconds / 20)

In [22]:
df.head(3)

,source.ip,destination.ip,destination.port,ts,conn_count,deltas,tsLow,tsMid,tsHigh,tsBowleyNum,tsBowleyDen,tsSkew,tsMadm,tsConnDiv
38,192.168.128.15,23.66.101.177,80,"[2021-12-29 22:20:52.436000+00:00, 2021-12-29 22:20:52.447000+00:00, 2021-12-29 22:20:52.464000+00:00, 2021-12-29 22:20:52.498000+00:00, 2021-12-29 22:20:52.498000+00:00, 2021-12-29 22:20:52.503000+00:00, 2021-12-29 22:20:52.516000+00:00, 2021-12-29 22:20:52.526000+00:00, 2021-12-29 22:20:52.552000+00:00, 2021-12-29 22:20:52.576000+00:00, 2021-12-29 22:20:52.576000+00:00, 2021-12-29 22:20:52.588000+00:00, 2021-12-29 22:20:52.593000+00:00, 2021-12-29 22:20:52.604000+00:00, 2021-12-29 22:20:52.621000+00:00, 2021-12-29 22:20:52.636000+00:00, 2021-12-29 22:20:52.651000+00:00, 2021-12-29 22:20:...",159,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...]",0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.15
129,192.168.128.50,192.168.120.158,80,"[2021-12-30 12:56:41.494000+00:00, 2021-12-30 12:57:41.627000+00:00, 2021-12-30 12:58:41.787000+00:00, 2021-12-30 12:59:41.807000+00:00, 2021-12-30 13:00:41.824000+00:00, 2021-12-30 13:01:41.845000+00:00, 2021-12-30 13:02:41.870000+00:00, 2021-12-30 13:03:41.893000+00:00, 2021-12-30 13:04:41.919000+00:00, 2021-12-30 13:05:41.933000+00:00, 2021-12-30 13:06:42.024000+00:00, 2021-12-30 13:07:42.042000+00:00, 2021-12-30 13:08:42.055000+00:00, 2021-12-30 13:09:42.071000+00:00, 2021-12-30 13:10:42.097000+00:00, 2021-12-30 13:11:42.119000+00:00, 2021-12-30 13:12:42.142000+00:00, 2021-12-30 13:13:...",2036,"[60.0, 60.0, 60.0, 60.0, 60.0, 60.0, 60.0, 60.0, 60.0, 60.0, 60.0, 60.0, 60.0, 60.0, 60.0, 60.0, 60.0, 60.0, 60.0, 60.0, 60.0, 60.0, 60.0, 60.0, 60.0, 60.0, 60.0, 60.0, 60.0, 60.0, 60.0, 60.0, 60.0, 15484.0, 60.0, 0.0, 5721.0, 60.0, 0.0, 60.0, 60.0, 0.0, 59.0, 0.0, 59.0, 0.0, 65.0, 54.0, 10.0, 65.0, 44.0, 20.0, 65.0, 34.0, 30.0, 65.0, 24.0, 40.0, 65.0, 14.0, 50.0, 65.0, 4.0, 60.0, 59.0, 5.0, 65.0, 49.0, 15.0, 65.0, 39.0, 25.0, 65.0, 29.0, 35.0, 65.0, 19.0, 45.0, 65.0, 9.0, 55.0, 64.0, 0.0, 65.0, 54.0, 10.0, 65.0, 44.0, 20.0, 65.0, 1451.0, 30.0, 65.0, 24.0, 40.0, 65.0, 14.0, 40.0, 9.0, 27.0...",52.0,87.0,106.0,-16.0,54.0,-0.296296,18.0,1125.10
130,192.168.128.50,192.168.120.158,443,"[2021-12-29 19:58:55.104000+00:00, 2021-12-29 20:00:55.398000+00:00, 2021-12-29 20:02:55.601000+00:00, 2021-12-29 20:04:55.813000+00:00, 2021-12-29 20:04:55.825000+00:00, 2021-12-29 20:06:56.198000+00:00, 2021-12-29 20:08:56.445000+00:00, 2021-12-29 20:10:56.640000+00:00, 2021-12-29 20:18:37.658000+00:00, 2021-12-29 20:18:57.668000+00:00, 2021-12-29 20:20:57.892000+00:00, 2021-12-29 20:22:58.180000+00:00, 2021-12-29 20:24:58.392000+00:00, 2021-12-29 20:26:58.640000+00:00, 2021-12-29 20:28:58.870000+00:00, 2021-12-29 20:30:59.125000+00:00, 2021-12-29 20:32:59.367000+00:00, 2021-12-29 20:34:...",72,"[120.0, 120.0, 120.0, 0.0, 120.0, 120.0, 120.0, 461.0, 20.0, 120.0, 120.0, 120.0, 120.0, 120.0, 120.0, 120.0, 120.0, 120.0, 120.0, 120.0, 120.0, 120.0, 120.0, 120.0, 120.0, 120.0, 120.0, 120.0, 120.0, 120.0, 120.0, 120.0, 120.0, 120.0, 120.0, 120.0, 120.0, 120.0, 120.0, 120.0, 120.0, 120.0, 120.0, 120.0, 120.0, 120.0, 120.0, 120.0, 120.0, 120.0, 120.0, 676.0, 20.0, 120.0, 120.0, 120.0, 120.0, 120.0, 100.0, 20.0, 0.0, 120.0, 120.0, 120.0, 120.0, 120.0, 120.0, 120.0, 120.0, 120.0, 120.0]",120.0,120.0,120.0,0.0,0.0,0.000000,0.0,443.60


In [23]:
df['tsSkewScore'] = 1.0 - abs(df['tsSkew'])
df['tsMadmScore'] = 1.0 - df['tsMadm']/30.0
df['tsMadmScore'] = df['tsMadmScore'].apply(lambda x: 0 if x < 0 else x)
df['tsConnCountScore'] = (df['conn_count']) / df['tsConnDiv']
df['tsConnCountScore'] = df['tsConnCountScore'].apply(lambda x: 1.0 if x > 1.0 else x)
df['tsScore'] = (((df['tsSkewScore'] + df['tsMadmScore'] + df['tsConnCountScore']) / 3.0) * 1000) / 1000
df.sort_values(by= 'tsScore', ascending=False, inplace=True, ignore_index=True)
df[columns_to_display].head(30)

,tsScore,conn_count,source.ip,destination.ip,destination.port,deltas
0,1.000000,159,192.168.128.15,23.66.101.177,80,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...]"
1,1.000000,159,192.168.128.50,8.240.196.126,80,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...]"
2,0.720769,72,192.168.128.50,192.168.120.158,443,"[120.0, 120.0, 120.0, 0.0, 120.0, 120.0, 120.0, 461.0, 20.0, 120.0, 120.0, 120.0, 120.0, 120.0, 120.0, 120.0, 120.0, 120.0, 120.0, 120.0, 120.0, 120.0, 120.0, 120.0, 120.0, 120.0, 120.0, 120.0, 120.0, 120.0, 120.0, 120.0, 120.0, 120.0, 120.0, 120.0, 120.0, 120.0, 120.0, 120.0, 120.0, 120.0, 120.0, 120.0, 120.0, 120.0, 120.0, 120.0, 120.0, 120.0, 120.0, 676.0, 20.0, 120.0, 120.0, 120.0, 120.0, 120.0, 100.0, 20.0, 0.0, 120.0, 120.0, 120.0, 120.0, 120.0, 120.0, 120.0, 120.0, 120.0, 120.0]"
3,0.720026,97,192.168.128.50,192.168.120.158,443,"[120.0, 120.0, 120.0, 120.0, 120.0, 120.0, 120.0, 120.0, 120.0, 120.0, 120.0, 120.0, 120.0, 120.0, 120.0, 120.0, 120.0, 120.0, 120.0, 120.0, 120.0, 120.0, 120.0, 120.0, 120.0, 120.0, 120.0, 120.0, 120.0, 120.0, 120.0, 120.0, 120.0, 120.0, 120.0, 120.0, 120.0, 120.0, 120.0, 120.0, 120.0, 120.0, 120.0, 120.0, 120.0, 120.0, 120.0, 120.0, 120.0, 120.0, 120.0, 120.0, 120.0, 120.0, 120.0, 120.0, 120.0, 120.0, 120.0, 120.0, 120.0, 120.0, 120.0, 120.0, 120.0, 120.0, 120.0, 120.0, 120.0, 120.0, 120.0, 120.0, 120.0, 120.0, 120.0, 120.0, 120.0, 120.0, 120.0, 696.0, 120.0, 120.0, 120.0, 120.0, 120.0, ..."
4,0.701235,2036,192.168.128.50,192.168.120.158,80,"[60.0, 60.0, 60.0, 60.0, 60.0, 60.0, 60.0, 60.0, 60.0, 60.0, 60.0, 60.0, 60.0, 60.0, 60.0, 60.0, 60.0, 60.0, 60.0, 60.0, 60.0, 60.0, 60.0, 60.0, 60.0, 60.0, 60.0, 60.0, 60.0, 60.0, 60.0, 60.0, 60.0, 15484.0, 60.0, 0.0, 5721.0, 60.0, 0.0, 60.0, 60.0, 0.0, 59.0, 0.0, 59.0, 0.0, 65.0, 54.0, 10.0, 65.0, 44.0, 20.0, 65.0, 34.0, 30.0, 65.0, 24.0, 40.0, 65.0, 14.0, 50.0, 65.0, 4.0, 60.0, 59.0, 5.0, 65.0, 49.0, 15.0, 65.0, 39.0, 25.0, 65.0, 29.0, 35.0, 65.0, 19.0, 45.0, 65.0, 9.0, 55.0, 64.0, 0.0, 65.0, 54.0, 10.0, 65.0, 44.0, 20.0, 65.0, 1451.0, 30.0, 65.0, 24.0, 40.0, 65.0, 14.0, 40.0, 9.0, 27.0..."


In [24]:
df.loc[df['tsScore'] > 0.80, columns_to_display]

,tsScore,conn_count,source.ip,destination.ip,destination.port,deltas
0,1.0,159,192.168.128.15,23.66.101.177,80,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...]"
1,1.0,159,192.168.128.50,8.240.196.126,80,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...]"
